In [1]:
import pandas as pd

import numpy as np

In [2]:

# srcDataFrame = pd.read_csv('optionsDX/aapl_2022/aapl_eod_2022q4-86egbl/aapl_eod_202210.txt')
srcDataFrame = pd.read_csv('tslaReduced.csv')

df = pd.DataFrame(srcDataFrame)
df.columns = df.columns.str.strip()

/var/folders/p_/r9zh86p106b8yh2ly8zzjbw00000gn/T/ipykernel_2313/155347403.py:2: DtypeWarning: Columns (4,5,6,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  srcDataFrame = pd.read_csv('tslaReduced.csv')


In [3]:
# df['[QUOTE_DATE]'] = df.get('[QUOTE_DATE]').str.strip()
# # df['[QUOTE_DATE]']

In [4]:
dayGroups = df.groupby('[QUOTE_DATE]')

In [5]:
print(type( dayGroups))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [6]:
# import OptionPlayTranslator
from OptionPlayTranslator import *
# from utils.options_utils import *

In [7]:

opt = OPT('AAPL', dayGroups)
print(opt.ticker)
opt.find_plays(' 2019-10-01', ' 2020-01-16','200')

AAPL
11209  chain size
(98, 11)  week slice shape
(189, 11)  week slice shape


,[QUOTE_DATE],[UNDERLYING_LAST],[EXPIRE_DATE],[DTE],[C_LAST],[C_BID],[C_ASK],[STRIKE],[P_BID],[P_ASK],[P_LAST]
238791,2019-10-01,244.56,2020-01-17,108.04,0.0,233.66,235.76,10.0,0.01,0.01,0.04
238792,2019-10-01,244.56,2020-01-17,108.04,0.0,228.66,230.76,15.0,0.01,0.02,0.02
238793,2019-10-01,244.56,2020-01-17,108.04,0.0,223.64,225.75,20.0,0.02,0.16,0.07
238794,2019-10-01,244.56,2020-01-17,108.04,0.0,218.65,220.75,25.0,0.03,0.16,0.09
238795,2019-10-01,244.56,2020-01-17,108.04,0.0,213.65,215.75,30.0,0.07,0.1,0.12
...,...,...,...,...,...,...,...,...,...,...,...
238884,2019-10-01,244.56,2020-01-17,108.04,0.0,0.0,0.06,660.0,414.0,416.55,0.0
238885,2019-10-01,244.56,2020-01-17,108.04,0.11,0.0,0.07,670.0,424.0,426.54,0.0
238886,2019-10-01,244.56,2020-01-17,108.04,0.1,0.0,0.17,680.0,433.94,436.56,0.0
238887,2019-10-01,244.56,2020-01-17,108.04,0.0,0.0,0.03,690.0,443.95,446.55,0.0


In [8]:
opt.chain.columns

opt.chain

,[QUOTE_DATE],[UNDERLYING_LAST],[EXPIRE_DATE],[DTE],[C_LAST],[C_BID],[C_ASK],[STRIKE],[P_BID],[P_ASK],[P_LAST]
238272,2019-10-01,244.56,2019-10-04,3.00,0.0,113.94,115.75,130.0,0.0,0.01,0.01
238273,2019-10-01,244.56,2019-10-04,3.00,0.0,108.99,111.11,135.0,0.0,0.01,0.01
238274,2019-10-01,244.56,2019-10-04,3.00,0.0,103.8,105.91,140.0,0.0,0.01,0.0
238275,2019-10-01,244.56,2019-10-04,3.00,0.0,98.6,101.06,145.0,0.0,0.02,0.02
238276,2019-10-01,244.56,2019-10-04,3.00,94.1,93.56,95.91,150.0,0.0,0.01,0.02
...,...,...,...,...,...,...,...,...,...,...,...
239286,2019-10-01,244.56,2022-01-21,843.04,0.0,31.5,37.5,360.0,140.5,150.05,0.0
239287,2019-10-01,244.56,2022-01-21,843.04,0.0,29.01,35.0,370.0,148.5,157.89,0.0
239288,2019-10-01,244.56,2022-01-21,843.04,30.03,27.01,33.0,380.0,156.0,165.5,163.75
239289,2019-10-01,244.56,2022-01-21,843.04,27.97,25.0,31.01,390.0,163.99,173.51,0.0


In [9]:
options = opt.pred_week
print(options.shape)

(98, 11)


In [10]:
print(opt.fut_friday)
ex = opt.chain.loc[opt.chain['[EXPIRE_DATE]'] == opt.fut_friday]
print(ex.size)

 2020-01-17
1078
